# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 10, 22, 12

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/4h, trade type is sell
Number of combinations is 75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [02:46<00:00,  2.23s/it]


# Check local statistics

In [3]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
24,STOCH_RSI,12,35,9,7,3,30,91.89,59.46,59.46,69.19,71.89,71.89,65.41,64.86,68.11,76.22,78.38,78.92,82.70,80.00,78.38,81.62,81.08,80.00,77.30,81.08,75.14,72.43,69.19,66.49,-0.51,-0.25,-0.18,-0.85,-1.38,-1.30,-0.77,-0.55,-0.70,-1.09,-1.19,-1.48,-1.60,-1.80,-2.00,-2.09,-2.35,-2.01,-1.92,-1.71,-1.35,-1.21,-1.06,-0.79,185,74.212083,-1.255833,779.235417,-232.329167
23,STOCH_RSI,12,35,9,7,3,25,91.67,60.56,57.78,68.33,71.11,71.11,64.44,65.56,68.89,75.56,78.89,78.33,82.78,79.44,77.78,81.11,80.56,80.00,77.22,81.11,75.56,72.22,69.44,66.67,-0.51,-0.26,-0.17,-0.74,-1.31,-1.28,-0.76,-0.53,-0.69,-1.10,-1.18,-1.48,-1.58,-1.77,-1.99,-2.06,-2.35,-2.01,-1.94,-1.72,-1.49,-1.21,-1.07,-0.83,180,74.005000,-1.251250,720.900000,-225.225000
22,STOCH_RSI,12,35,9,7,3,20,90.60,62.42,57.05,68.46,71.81,71.14,63.76,63.76,68.46,75.84,78.52,78.52,83.22,81.21,79.87,83.22,81.88,81.21,78.52,81.21,74.50,70.47,67.11,65.10,-0.51,-0.34,-0.14,-0.78,-1.32,-1.24,-0.76,-0.55,-0.70,-1.08,-1.13,-1.43,-1.50,-1.80,-2.00,-2.09,-2.35,-2.13,-1.99,-1.83,-1.53,-1.26,-1.02,-0.88,149,74.077500,-1.265000,607.547500,-188.485000
17,STOCH_RSI,12,30,9,7,3,20,86.79,58.49,66.04,71.70,75.47,73.58,75.47,69.81,69.81,77.36,75.47,75.47,83.02,83.02,81.13,84.91,84.91,84.91,81.13,79.25,73.58,69.81,69.81,62.26,-0.51,-0.40,-0.31,-1.24,-1.30,-1.51,-1.10,-0.72,-0.73,-1.16,-1.32,-1.35,-1.84,-1.98,-2.15,-2.45,-2.07,-2.13,-2.04,-1.99,-1.56,-1.24,-1.19,-0.79,53,75.550000,-1.378333,294.150000,-73.051667
19,STOCH_RSI,12,30,9,7,3,30,87.93,56.90,67.24,72.41,75.86,74.14,75.86,68.97,68.97,77.59,74.14,75.86,82.76,81.03,79.31,84.48,81.03,81.03,77.59,79.31,74.14,68.97,67.24,60.34,-0.49,-0.38,-0.33,-1.16,-1.24,-1.30,-1.05,-0.69,-0.74,-1.19,-1.27,-1.36,-1.64,-1.93,-2.11,-2.42,-2.02,-2.07,-1.99,-1.81,-1.31,-1.02,-1.13,-0.78,58,74.712500,-1.309583,273.325000,-75.955833
47,STOCH_RSI,14,35,9,7,3,20,85.71,58.10,57.14,71.43,72.38,71.43,65.71,66.67,67.62,74.29,77.14,78.10,83.81,80.95,79.05,81.90,80.00,78.10,74.29,77.14,70.48,67.62,62.86,59.05,-0.51,-0.26,-0.24,-0.96,-1.38,-1.24,-0.73,-0.56,-0.70,-1.01,-1.01,-1.38,-1.50,-1.76,-1.98,-1.98,-2.18,-2.02,-1.96,-1.72,-1.26,-1.19,-0.71,-0.62,105,72.540417,-1.202500,266.743750,-126.262500
48,STOCH_RSI,14,35,9,7,3,25,87.20,57.60,58.40,68.80,70.40,70.40,64.80,67.20,67.20,74.40,78.40,79.20,82.40,77.60,76.00,78.40,76.80,76.00,72.80,77.60,72.00,69.60,66.40,60.00,-0.51,-0.26,-0.28,-0.88,-1.24,-1.21,-0.73,-0.56,-0.70,-1.05,-1.13,-1.43,-1.50,-1.72,-1.87,-1.90,-2.08,-1.88,-1.80,-1.57,-1.26,-1.13,-0.92,-0.57,125,72.066667,-1.174167,258.333333,-146.770833
18,STOCH_RSI,12,30,9,7,3,25,87.50,57.14,66.07,71.43,75.00,73.21,75.00,69.64,69.64,76.79,75.00,75.00,82.14,80.36,78.57,83.93,80.36,80.36,76.79,78.57,73.21,69.64,69.64,62.50,-0.45,-0.38,-0.33,-1.02,-1.24,-1.31,-1.05,-0.69,-0.74,-1.19,-1.27,-1.28,-1.54,-1.83,-2.06,-2.17,-1.89,-2.07,-1.99,-1.97,-1.46,-1.18,-1.21,-0.83,56,74.478750,

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            

# Save new config data to config file

In [7]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                            'slowd_period': [3], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)